In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
from scipy import stats 
import scipy.stats as stats
import statsmodels.api as sm
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.ensemble import ExtraTreesRegressor,RandomForestRegressor
import os
import gc
import pylab
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
data_train = pd.read_csv('/content/train.csv')
data_test = pd.read_csv('/content/test.csv')
data_train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,B,C,E,A,C,T,B,G,A,A,I,E,G,J,G,BU,BC,C,AS,S,A,O,LB,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,L,F,A,A,E,E,I,K,K,BI,CQ,A,AV,BM,A,O,DP,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,B,B,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,...,A,D,D,C,E,E,A,D,L,O,A,B,E,F,H,F,A,AB,DK,A,C,AF,A,I,GK,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,D,D,C,E,E,D,T,I,D,A,A,E,E,I,K,K,BI,CS,C,N,AE,A,O,DJ,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,B,A,B,A,A,A,A,A,A,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,H,D,B,D,E,E,A,P,F,J,A,A,D,E,K,G,B,H,C,C,Y,BM,A,K,CK,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [3]:
X = data_train.drop(['loss'],axis = 1)
y = data_train['loss']

**Train-Test split**

In [4]:
X_train,X_test,y_train,y_test =  train_test_split(X,y,test_size = 0.2,random_state = 100)

print('shape of x_train : ',X_train.shape)
print('shape of y_train : ',y_train.shape)
print('shape of x_test  : ',X_test.shape)
print('shape of y_test  : ',y_test.shape)

shape of x_train :  (150654, 131)
shape of y_train :  (150654,)
shape of x_test  :  (37664, 131)
shape of y_test  :  (37664,)


In [5]:
print('Null value in train data: ',X_train.isnull().sum().sum())
print('Null value in test data: ',X_test.isnull().sum().sum())

Null value in train data:  0
Null value in test data:  0


In [6]:
categorical_features_train = list(X_train.select_dtypes(include=['object']).columns)
print("Categorical features in Train data : {} features".format(len(categorical_features_train)))
continous_features_train = list(X_train.select_dtypes(include=['float']).columns)
print("Continous features in Train data : {} features".format(len(continous_features_train)))

Categorical features in Train data : 116 features
Continous features in Train data : 14 features


In [7]:
categorical_features_test = list(X_test.select_dtypes(include=['object']).columns)
print("Categorical features in Test data : {} features".format(len(categorical_features_test)))
continous_features_test = list(X_test.select_dtypes(include=['float']).columns)
print("Continous features in Test data : {} features".format(len(continous_features_test)))

Categorical features in Test data : 116 features
Continous features in Test data : 14 features


In [8]:
'''
Here we creating Dictionary for catehories and there frequency of levels in each categorical features
'''
feature_count_train_dict = {i : dict(X_train.groupby(i)['id'].count()) for i in categorical_features_train}

### **Features in test and train have different levels**

In [11]:
mismatch_cat_features=[]
for col in categorical_features_train:
  if (set(feature_count_train_dict[col].keys()) - set(X_test[col].unique())) or (set(X_test[col].unique()) - set(feature_count_train_dict[col].keys())):
    mismatch_cat_features.append(col)
print('Features that have mismatch of levels in train and test set are :',mismatch_cat_features)
print('Number of categorical features that has mismatch in levels are: ',len(mismatch_cat_features))

Features that have mismatch of levels in train and test set are : ['cat75', 'cat89', 'cat90', 'cat92', 'cat96', 'cat99', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat109', 'cat110', 'cat111', 'cat113', 'cat114', 'cat115', 'cat116']
Number of categorical features that has mismatch in levels are:  19


In [12]:
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

### **Different levels are replaced by 'NaN' in test data**

In [13]:
'''
Handling mismatch level in features .

Simple way is to put 'NaN' if we found unknown level in test set.

Like 'cat101' has levels ---> A,B,C in train set and has levels ---> A,B,D.Now level D is present in test set but not in train set so level D is unknown.So we replace level D in with NaN.

So finally in test set 'cat101' has levels ---> A,B,NaN.

So we have to handle these NaN values in test set while Encoding categorical features.
'''

def search_feature(x,combine_set):
  #print(combine_set)
  if x in combine_set:
      return np.nan
  return x


for col in categorical_features_test:
  if (set(feature_count_train_dict[col].keys()) - set(X_test_copy[col].unique())) or (set(X_test_copy[col].unique()) - set(feature_count_train_dict[col].keys())) :
    #print('yes')
    unique_train = set(feature_count_train_dict[col].keys())
    unique_test = set(X_test_copy[col].unique())
    rem_train = unique_train-unique_test
    rem_test = unique_test-unique_train
    #print('rem_train',rem_train)
    #print('rem_test',rem_test)
    combine_set = rem_train.union(rem_test)
    #print('combine_set',combine_set)
    mapp_nan = {i: np.nan for i in combine_set }
    #print( mapp_nan)
    #uniq_nan_testdf['cat92'] = uniq_nan_testdf['cat92'].map(mapp_nan)

    X_test_copy[col] = X_test_copy[col].apply(lambda x: search_feature(x,combine_set),1)

In [14]:
#Is any null or nan in test data?

print('Number of NaN values in test set are : ', X_test_copy.isnull().sum().sum())


Number of NaN values in test set are :  24


In [15]:
X_test_copy[X_test_copy['cat110'].isnull()]

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
24699,77741,A,B,A,A,A,A,A,A,B,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,G,H,D,F,D,E,C,A,P,G,I,A,A,H,D,G,E,D,AB,NaN,A,AP,J,A,J,NaN,0.580885,0.005742,0.045491,0.267727,0.281143,0.412757,0.421691,0.29758,0.47481,0.45017,0.231253,0.227963,0.258586,0.354729
13636,43369,B,A,A,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,A,C,B,C,E,C,D,T,A,A,A,A,G,E,H,K,K,BI,NaN,A,AD,AX,O,Q,HG,0.325401,0.100099,0.187583,0.402922,0.281143,0.491827,0.480397,0.54236,0.38249,0.29595,0.334828,0.328268,0.636453,0.793193
119120,372165,A,B,A,A,B,A,A,A,B,B,A,B,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,A,A,...,B,H,D,B,C,E,C,D,T,F,M,A,B,G,E,H,K,K,BI,NaN,A,C,AX,A,Q,HG,0.325401,0.100099,0.187583,0.402922,0.281143,0.491827,0.480397,0.54236,0.38249,0.29595,0.334828,0.328268,0.636453,0.771694
15676,49769,B,A,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,A,B,A,B,A,A,A,A,A,A,A,A,A,A,A,A,A,A,...,A,A,D,B,C,E,E,A,K,I,A,A,A,D,D,I,E,A,AB,NaN,E,AI,X,A,I,CR,0.330514,0.036553,0.317666,0.886774,0.281143,0.233061,0.223816,0.61229,0.31581,0.34928,0.178160,0.185520,0.181433,0.345990


### **Regularized Target Encoding**

In [16]:
#https://medium.com/@pouryaayria/k-fold-target-encoding-dfe9a594874b
#https://www.kaggle.com/vprokopev/mean-likelihood-encodings-a-comprehensive-study
#https://www.kaggle.com/vchoubey/random-forest-mean-encoding-for-app-channel


'''
Regularization:

label(c) = (pc∗nc + pglobal∗α)/(nc+α)

pc :  is a target mean for a category
nc : is a number of samples in a category
pglobal :  is a global target mean
α : is a regularisation parameter that can be viewed as a size of a category you can trust
'''

from sklearn.model_selection import KFold


def mean_train_encoding_trial(X,y, cols, target,global_target_mean,alpha):
    df = X.join(y)
    df_copy = df.copy()

    skf = KFold(5, shuffle = True, random_state=123)
  
    for col in cols:
        df[col + '_mean_encoded'] = np.nan


    for trn_ind , val_ind in skf.split(df,y):
        x_tr, x_val =  df_copy.iloc[trn_ind], df_copy.iloc[val_ind]

        for col in cols:
            tr_mean = x_tr.groupby(col)[target].agg(['count','mean'])
            a = (tr_mean['count']*tr_mean['mean'])+(global_target_mean*alpha)
            b = (tr_mean['count'] + alpha)
            encoded = a/b
            mean = x_val[col].map(encoded)
            df_copy[col].iloc[val_ind] = mean
            df[col + '_mean_encoded'].iloc[val_ind] = mean

    prior = global_target_mean

    for col in cols:
        df_copy[col].fillna(prior, inplace = True) 
        df[col + '_mean_encoded'].fillna(prior, inplace = True) 
        
    return df_copy,df

In [17]:
#y_train = data['log_loss']
global_target_mean = y_train.mean()
cols = categorical_features_train
target = 'loss'
alpha = 5  #regularisation parameter
X_train_copy1 = X_train.copy()
final_X_train,df_train = mean_train_encoding_trial(X_train_copy1,y_train, cols, target,global_target_mean,alpha)


In [18]:
print(final_X_train.head(5))
print(df_train.head(5))

            id         cat1         cat2  ...    cont13    cont14     loss
180302  562558  3039.880401  3039.880401  ...  0.261150  0.827278  4641.62
105000  327963  3412.375872  3809.227474  ...  0.369740  0.283699  3466.19
179008  558531  3406.138609  2450.924557  ...  0.315758  0.364535  3327.13
121362  379025  3039.880401  3039.880401  ...  0.642600  0.390600  4052.56
82311   257441  3039.880401  3039.880401  ...  0.310026  0.212656  1287.14

[5 rows x 132 columns]
            id cat1  ... cat115_mean_encoded cat116_mean_encoded
180302  562558    A  ...         3039.880401         3039.880401
105000  327963    A  ...         3004.331697         2918.398592
179008  558531    A  ...         3008.199002         2913.973903
121362  379025    A  ...         3039.880401         3039.880401
82311   257441    A  ...         3039.880401         3039.880401

[5 rows x 248 columns]


In [19]:
'''
crearing mapping dict for test data encoding using train data.
'''

target_encoding_map_with_nan = {}

for col in categorical_features_train:
  z = df_train.groupby(col)[col + '_mean_encoded'].mean().to_dict()
  z[np.nan] = global_target_mean
  target_encoding_map_with_nan[col] = z

**Transform Test data**

In [20]:
X_test_cp1 = X_test_copy.copy()
for i,col in enumerate(categorical_features_test):
  X_test_cp1[col] =  X_test_cp1[col].map(target_encoding_map_with_nan[col])

X_test_cp1.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
76445,239095,2136.562139,2564.940050,2932.434117,3396.735601,2869.066188,2621.033370,2937.755268,2993.869798,2611.608353,2761.536845,2854.751307,2729.72068,2865.754979,3011.43902,3039.734701,2956.15757,3024.649598,3032.752258,3032.684655,3037.564156,3039.136991,3039.622307,2871.54988,2998.176892,2971.543844,3011.561872,3081.371914,2979.188008,3014.073217,3024.98491,3033.816409,3035.97278,3033.142619,3035.986936,3038.029793,2879.96303,3030.872268,2966.620223,3033.715628,...,2983.266936,3040.709524,3030.207379,1971.732987,3002.162795,3032.91331,3014.719749,3056.707029,3004.510428,2014.253674,2564.940050,2993.869798,2869.066188,2987.471386,2686.602694,2850.094000,3123.121399,2878.219197,2447.500031,2758.085390,3095.064123,2912.624696,2750.159044,2247.489761,3130.826026,3039.880401,0.238993,0.036553,0.129646,0.796346,0.281143,0.176477,0.170425,0.51853,0.24180,0.21230,0.126256,0.125933,0.204240,0.838263
137451,428873,3336.620917,3651.738149,2932.434117,3396.735601,2869.066188,3217.333933,7271.052652,2993.869798,3674.265445,2761.536845,2854.751307,2729.72068,2865.754979,3011.43902,3039.734701,2956.15757,3024.649598,3032.752258,3032.684655,3037.564156,3039.136991,3039.622307,2871.54988,2998.176892,2971.543844,3011.561872,3081.371914,2979.188008,3014.073217,3024.98491,3033.816409,3035.97278,3033.142619,3035.986936,3038.029793,2879.96303,3030.872268,2966.620223,3033.715628,...,3102.116834,3028.022626,3030.207379,3203.746644,3002.162795,3032.91331,2978.998062,3056.707029,3004.510428,3181.686566,2835.485457,2993.869798,2869.066188,2987.471386,2914.864827,2854.525940,3123.121399,2878.219197,3126.615122,3116.679380,3095.064123,3040.186654,3149.134733,3217.333933,3052.778335,3007.715397,0.356663,0.737068,0.634224,0.373816,0.491114,0.272540,0.295775,0.50060,0.33611,0.31480,0.257148,0.264760,0.339244,0.322202
168987,527062,3336.620917,3651.738149,2932.434117,2871.932500,2869.066188,3217.333933,2937.755268,2993.869798,3674.265445,2761.536845,2854.751307,2729.72068,2865.754979,3011.43902,3039.734701,2956.15757,3024.649598,3032.752258,3032.684655,3037.564156,3039.136991,3039.622307,2871.54988,2998.176892,2971.543844,3011.561872,3034.986010,2979.188008,3014.073217,3024.98491,3033.816409,3035.97278,3033.142619,3035.986936,3038.029793,2879.96303,3030.872268,2966.620223,3033.715628,...,3102.116834,3028.022626,3030.207379,3203.746644,3002.162795,3032.91331,3092.429526,3056.707029,3004.510428,3181.686566,2835.485457,2993.869798,2869.066188,2992.548985,2914.864827,2941.860829,3067.272863,2878.219197,3126.615122,3408.767801,2871.932500,3366.014296,2778.748154,3217.333933,3006.159805,2932.663067,0.513457,0.737068,0.549770,0.594598,0.811271,0.416181,0.408773,0.62918,0.50630,0.37493,0.338312,0.331643,0.315758,0.254741
55044,172381,3336.620917,2564.940050,2932.434117,2871.932500,2869.066188,3217.333933,2937.755268,2993.869798,2611.608353,2761.536845,2854.751307,2729.72068,2865.754979,3011.43902,3039.734701,2956.15757,3024.649598,3032.752258,3032.684655,3037.564156,3039.136991,3039.622307,2871.54988,2998.176892,2971.543844,3011.561872,3034.986010,2979.188008,3014.073217,3024.98491,3033.816409,3035.97278,3033.142619,3035.986936,3038.029793,2879.96303,3030.872268,2966.620223,3033.715628,...,2732.091187,3028.022626,3092.125292,3203.746644,2783.221684,3032.91331,3006.385724,2630.078725,2630.275209,2798.370315,2564.940050,2993.869798,2869.066188,3105.401600,3064.326068,3232.886153,2939.889073,2630.078725,3126.615122,2959.548737,2871.932500,3373.058512,2946.332834,3217.333933,30

In [21]:
print('Number of NaN in encoded test set are : ',X_test_cp1.isnull().sum().sum())


Number of NaN in encoded test set are :  0


In [22]:
#sanity check for NaN replacement
X_test_cp1['cat110'].loc[24699]

3039.8804011178336

**2. Frequency + Target_mean encoding**

In [23]:
'''
Now we seen we have some categorical feature with low cardinality and some vary high cardinality.

So we can do frequency label encoding for low cardinal features and target encoding for high cardinal features.

So we choose upto 4 levels feature are for label encoding and beyond 4 level for target encoding.
'''
greatrer_than_4_levels_features = []
less_than_4_levels_features = []
for col in categorical_features_train:
  cnt = len(feature_count_train_dict[col])
  if cnt > 4:
    greatrer_than_4_levels_features.append(col)
  else:
    less_than_4_levels_features.append(col)
print('Greatrer than 4 levels features are : ',greatrer_than_4_levels_features)
print('Numers of features that have levels more than 4 are : ',len(greatrer_than_4_levels_features))

Greatrer than 4 levels features are :  ['cat89', 'cat90', 'cat91', 'cat92', 'cat93', 'cat94', 'cat95', 'cat96', 'cat97', 'cat98', 'cat99', 'cat100', 'cat101', 'cat102', 'cat103', 'cat104', 'cat105', 'cat106', 'cat107', 'cat108', 'cat109', 'cat110', 'cat111', 'cat112', 'cat113', 'cat114', 'cat115', 'cat116']
Numers of features that have levels more than 4 are :  28


In [24]:
'''
We will create mapping dictonary to map lavel to their respective numeric label.
like in 'cat1' ---> A :1 ,B:0
assume Level 'A' is very frequent so we assign label 1.

like this we assign higher order to high frequent level in respective categorical feature.

And we also take 'Nan' case in consideration. we assigned -1 to 'nan'

'''
less_than_4_level_dict_map_with_nan={}
for col in less_than_4_levels_features:
  z = {k: i for i, k in enumerate(X_train.groupby([col])['id'].count().sort_values().index)}
  z[np.nan] = -1
  less_than_4_level_dict_map_with_nan[col]=z

In [25]:
print('Mapping dict for label encoding on feature cat88 are: ',less_than_4_level_dict_map_with_nan['cat88'])


Mapping dict for label encoding on feature cat88 are:  {'B': 0, 'E': 1, 'D': 2, 'A': 3, nan: -1}


**Regularized Target encoding for feature have greater than 4 level**

In [26]:
#y_train = data['log_loss']
global_target_mean = y_train.mean()
cols = greatrer_than_4_levels_features
target = 'loss'
alpha = 5  #regularisation parameter
X_train_copy = X_train.copy()
final_X_train1,df_train1 = mean_train_encoding_trial(X_train_copy,y_train, cols, target,global_target_mean,alpha)


In [27]:
print(final_X_train1)
print(df_train1)

            id cat1 cat2 cat3  ...    cont12    cont13    cont14     loss
180302  562558    A    B    B  ...  0.178698  0.261150  0.827278  4641.62
105000  327963    A    B    A  ...  0.345316  0.369740  0.283699  3466.19
179008  558531    A    A    A  ...  0.301921  0.315758  0.364535  3327.13
121362  379025    A    B    A  ...  0.519456  0.642600  0.390600  4052.56
82311   257441    A    B    A  ...  0.264760  0.310026  0.212656  1287.14
...        ...  ...  ...  ...  ...       ...       ...       ...      ...
82270   257315    B    A    A  ...  0.594646  0.660756  0.806916   257.24
65615   205512    B    B    A  ...  0.744640  0.782450  0.475135   519.23
77655   243040    A    B    A  ...  0.387819  0.645656  0.433689  2150.77
56088   175794    A    A    A  ...  0.523266  0.318646  0.208077  1039.43
38408   120400    A    B    B  ...  0.255941  0.315758  0.478452  6328.48

[150654 rows x 132 columns]
            id cat1  ... cat115_mean_encoded cat116_mean_encoded
180302  562558    

In [28]:

target_encoding_map_morethan_4_level_with_nan = {}
for col in greatrer_than_4_levels_features:
  z =  df_train1.groupby(col)[col + '_mean_encoded'].mean().to_dict()
  z[np.nan] = global_target_mean
  target_encoding_map_morethan_4_level_with_nan[col] = z

In [29]:
print('Target encoded level mapping for cat105 feature are : ',target_encoding_map_morethan_4_level_with_nan['cat105'])


Target encoded level mapping for cat105 feature are :  {'A': 2871.380322505312, 'B': 2766.423038419884, 'C': 2850.632583091032, 'D': 2686.6026942726958, 'E': 2914.8648268839183, 'F': 3064.3260682790337, 'G': 3182.911832458204, 'H': 3346.9182147282895, 'I': 4113.432095410563, 'J': 4475.090195820596, 'K': 4629.398888094345, 'L': 4754.42749287419, 'M': 5200.7480910182185, 'N': 5605.358130180496, 'O': 4666.801750775769, 'P': 10964.461520213274, 'Q': 3481.7928676913475, 'R': 3039.8804011178336, 'S': 3039.8804011178336, nan: 3039.8804011178336}


In [30]:
'''
Here we applying Label encoding for lower cardinal features.So we wil get complete encoding.
'''
for col in less_than_4_levels_features:
  final_X_train1[col] = final_X_train1[col].map(less_than_4_level_dict_map_with_nan[col])

final_X_train1.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
180302,562558,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,0.289618,0.358319,0.506105,0.354893,0.302678,0.182014,0.308785,0.29758,0.32128,0.21230,0.180456,0.178698,0.261150,0.827278,4641.62
105000,327963,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,3044.626254,3031.594290,3247.619321,2998.696247,3033.652644,3118.928694,3067.698806,3001.323425,4737.665787,5248.999609,2985.538983,2828.685885,2993.294803,2881.331979,2937.539341,3136.476995,2839.870795,2598.375593,3423.281958,3099.091900,3095.313017,2932.426673,3264.709118,3004.331697,2918.398592,0.500412,0.681761,0.592681,0.738221,0.413817,0.399144,0.360918,0.60087,0.43524,0.45567,0.348867,0.345316,0.369740,0.283699,3466.19
179008,558531,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,3014.367565,3019.179246,3243.245024,2987.823271,3024.948681,3089.863567,3058.458459,2991.008509,3198.961898,2450.924557,2975.198814,2821.497140,2978.625822,2871.286502,2812.762379,3067.266439,2830.915961,3145.891485,2926.335798,3117.395424,3224.654146,2914.982615,3257.132056,3008.199002,2913.973903,0.480125,0.299102,0.484196,0.373816,0.422268,0.301647,0.360121,0.26847,0.41471,0.43919,0.307628,0.301921,0.315758,0.364535,3327.13
121362,379025,1,0,1,1,0,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,...,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,0.548107,0.299102,0.613660,0.383428,0.732367,0.563226,0.454015,0.61229,0.54609,0.54433,0.492200,0.519456,0.642600,0.390600,4052.56
82311,257441,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,3039.880401,0.470002,0.159990,0.419161,0.336414,0.643315,0.287716,0.357736,0.26847,0.39447,0.37493,0.260140,0.264760,0.310026,0.212656,1287.14


**Encoding for test data**

In [31]:
def test_label_plus_target_encoding(df_test,greatrer_than_4_levels_features,less_than_4_levels_features,target_encoding_map_morethan_4_level_with_nan,less_than_4_level_dict_map_with_nan):
  
  for col in greatrer_than_4_levels_features:
    df_test[col] = df_test[col].map( target_encoding_map_morethan_4_level_with_nan[col])

  for col in less_than_4_levels_features:
    df_test[col] = df_test[col].map(less_than_4_level_dict_map_with_nan[col])

  #final_test_data = test_data.drop(greatrer_than_4_levels_features,axis = 1)

  return df_test

In [32]:
X_test_cp2 = X_test_copy.copy()
final_X_test_data = test_label_plus_target_encoding(X_test_cp2,greatrer_than_4_levels_features,
                                                  less_than_4_levels_features,
                                                  target_encoding_map_morethan_4_level_with_nan,
                                                  less_than_4_level_dict_map_with_nan)

final_X_test_data.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
76445,239095,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...,2983.266936,3040.709524,3030.207379,1971.732987,3002.162795,3032.91331,3014.719749,3056.707029,3004.510428,2014.253674,2564.940050,2993.869798,2869.066188,2987.471386,2686.602694,2850.094000,3123.121399,2878.219197,2447.500031,2758.085390,3095.064123,2912.624696,2750.159044,2247.489761,3130.826026,3039.880401,0.238993,0.036553,0.129646,0.796346,0.281143,0.176477,0.170425,0.51853,0.24180,0.21230,0.126256,0.125933,0.204240,0.838263
137451,428873,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...,3102.116834,3028.022626,3030.207379,3203.746644,3002.162795,3032.91331,2978.998062,3056.707029,3004.510428,3181.686566,2835.485457,2993.869798,2869.066188,2987.471386,2914.864827,2854.525940,3123.121399,2878.219197,3126.615122,3116.679380,3095.064123,3040.186654,3149.134733,3217.333933,3052.778335,3007.715397,0.356663,0.737068,0.634224,0.373816,0.491114,0.272540,0.295775,0.50060,0.33611,0.31480,0.257148,0.264760,0.339244,0.322202
168987,527062,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,3102.116834,3028.022626,3030.207379,3203.746644,3002.162795,3032.91331,3092.429526,3056.707029,3004.510428,3181.686566,2835.485457,2993.869798,2869.066188,2992.548985,2914.864827,2941.860829,3067.272863,2878.219197,3126.615122,3408.767801,2871.932500,3366.014296,2778.748154,3217.333933,3006.159805,2932.663067,0.513457,0.737068,0.549770,0.594598,0.811271,0.416181,0.408773,0.62918,0.50630,0.37493,0.338312,0.331643,0.315758,0.254741
55044,172381,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,2732.091187,3028.022626,3092.125292,3203.746644,2783.221684,3032.91331,3006.385724,2630.078725,2630.275209,2798.370315,2564.940050,2993.869798,2869.066188,3105.401600,3064.326068,3232.886153,2939.889073,2630.078725,3126.615122,2959.548737,2871.932500,3373.058512,2946.332834,3217.333933,3012.235822,3119.915328,0.642763,0.422197,0.613660,0.327354,0.422268,0.688705,0.480915,0.33906,0.62542,0.73106,0.661688,0.648446,0.687115,0.211274
72647,227026,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,2732.091187,3028.022626,3030.207379,2702.691099,3002.162795,3032.91331,3020.495416,3056.707029,3004.510428,2184.311518,2564.940050,2993.869798,2869.066188,3105.401600,3182.911832,3232.886153,3067.272863,2878.219197,3126.615122,3116.679380,2871.932500,3170.617978,2519.516973,3534.612627,3130.826026,3119.915328,0.585115,0.555782,0.571360,0.554673,0.594196,0.502705,0.480792,0.43518,0.66201,0.58257,0.644013,0.630853,0.354344,0.824170


In [34]:
print('Number of NaN in encoded test data are : ',final_X_test_data.isnull().sum().sum())


Number of NaN in encoded test data are :  0


In [ ]:
for col in categorical_features_test:
  print(final_X_test_data[final_X_test_data[col].isnull()].index.tolist(),col)

## **Frequency label Encoding**

Here we will do frequency based label encoding as we do for low level features.

Here we will generate unique labels on levels for categorical features.

E.g. let 'cat1' feature has 4 levels with frequency are A>C>B>D. that is A has occured more time so will 
      assigned highest label to A level. After encoding we get  D:0,B:1,C:2,A:4. This way we will assigned labels.

      Since we have NaN in test set so we have handle that as well.We will assigned -1 label to nan. 

In [35]:
label_encoded_map_dict = {}
for col in categorical_features_train:
  z = {k: i for i, k in enumerate(X_train.groupby([col])['id'].count().sort_values().index)}
  z[np.nan] = -1
  label_encoded_map_dict[col]=z

In [36]:
label_encoded_map_dict['cat80']

{'A': 0, 'B': 2, 'C': 1, 'D': 3, nan: -1}

In [37]:
X_train_label_encoded = X_train.copy()
for col in categorical_features_train:
  X_train_label_encoded[col] = X_train[col].map(label_encoded_map_dict[col])
X_train_label_encoded.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
180302,562558,1,0,0,1,1,1,1,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,7,5,3,5,4,7,5,4,12,12,15,8,12,9,18,10,18,5,80,124,15,29,57,18,17,279,0.289618,0.358319,0.506105,0.354893,0.302678,0.182014,0.308785,0.29758,0.32128,0.21230,0.180456,0.178698,0.261150,0.827278
105000,327963,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,7,5,4,6,3,7,6,4,15,10,10,8,12,16,18,15,18,10,82,85,14,45,36,18,22,309,0.500412,0.681761,0.592681,0.738221,0.413817,0.399144,0.360918,0.60087,0.43524,0.45567,0.348867,0.345316,0.369740,0.283699
179008,558531,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,6,4,4,6,3,7,6,4,15,14,18,8,12,16,18,13,19,10,83,124,14,40,57,18,22,309,0.480125,0.299102,0.484196,0.373816,0.422268,0.301647,0.360121,0.26847,0.41471,0.43919,0.307628,0.301921,0.315758,0.364535
121362,379025,1,0,1,1,0,1,1,1,0,0,1,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,...,7,5,3,4,2,7,4,1,13,12,10,8,11,13,17,16,14,6,83,88,15,47,58,18,21,306,0.548107,0.299102,0.613660,0.383428,0.732367,0.563226,0.454015,0.61229,0.54609,0.54433,0.492200,0.519456,0.642600,0.390600
82311,257441,1,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,7,5,4,5,3,7,6,4,15,13,16,8,12,16,18,13,19,10,80,124,14,46,57,18,22,249,0.470002,0.159990,0.419161,0.336414,0.643315,0.287716,0.357736,0.26847,0.39447,0.37493,0.260140,0.264760,0.310026,0.212656


In [41]:
print('Number of NaN after encoding train set are :',X_train_label_encoded.isnull().sum().sum())


Number of NaN after encoding train set are : 0


**Encoding Test data**

In [39]:
X_test_label_encoded = X_test_copy.copy()

for col in categorical_features_test:
  X_test_label_encoded [col] = X_test_copy[col].map(label_encoded_map_dict[col])
X_test_label_encoded .head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,cat11,cat12,cat13,cat14,cat15,cat16,cat17,cat18,cat19,cat20,cat21,cat22,cat23,cat24,cat25,cat26,cat27,cat28,cat29,cat30,cat31,cat32,cat33,cat34,cat35,cat36,cat37,cat38,cat39,...,cat91,cat92,cat93,cat94,cat95,cat96,cat97,cat98,cat99,cat100,cat101,cat102,cat103,cat104,cat105,cat106,cat107,cat108,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
76445,239095,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...,7,5,4,3,3,7,2,4,15,7,18,8,12,14,15,10,18,10,80,121,14,35,29,16,20,2,0.238993,0.036553,0.129646,0.796346,0.281143,0.176477,0.170425,0.51853,0.24180,0.21230,0.126256,0.125933,0.204240,0.838263
137451,428873,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,...,6,4,4,6,3,7,5,4,15,14,16,8,12,14,18,13,18,10,83,123,14,19,44,18,17,305,0.356663,0.737068,0.634224,0.373816,0.491114,0.272540,0.295775,0.50060,0.33611,0.31480,0.257148,0.264760,0.339244,0.322202
168987,527062,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,6,4,4,6,3,7,6,4,15,14,16,8,12,16,18,16,19,10,83,127,15,49,53,18,22,309,0.513457,0.737068,0.549770,0.594598,0.811271,0.416181,0.408773,0.62918,0.50630,0.37493,0.338312,0.331643,0.315758,0.254741
55044,172381,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,5,4,3,6,2,7,3,1,13,11,18,8,12,15,17,14,14,6,83,124,15,46,57,18,19,310,0.642763,0.422197,0.613660,0.327354,0.422268,0.688705,0.480915,0.33906,0.62542,0.73106,0.661688,0.648446,0.687115,0.211274
72647,227026,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,...,5,4,4,5,3,7,4,4,15,13,18,8,12,15,16,14,19,10,83,123,15,50,34,13,20,310,0.585115,0.555782,0.571360,0.554673,0.594196,0.502705,0.480792,0.43518,0.66201,0.58257,0.644013,0.630853,0.354344,0.824170


In [42]:
print('Number of NaN in encoded test data are : ',X_test_label_encoded.isnull().sum().sum())


Number of NaN in encoded test data are :  0


Simillarly we can do encoding with log_transformed loss.